# Create tables and insert rows

In [ ]:
from cassandra.cluster import Cluster

In [ ]:
cluster = Cluster(['localhost'])

In [ ]:
session = cluster.connect()

In [ ]:
session.execute("CREATE KEYSPACE stocks \
                   WITH replication = {'class':'SimpleStrategy', \
                                        'replication_factor':'1'}")

In [ ]:
session.set_keyspace('stocks')

In [ ]:
session.execute("""
                  CREATE TABLE company (
                      company_id text,
                      name_latest text,
                      name_previous text,
                      PRIMARY KEY (company_id)
                   )
                """)

In [ ]:
session.execute("""
                  CREATE TABLE indicator_by_company (
                      company_id text,
                      indicator_id text,
                      yr_2010 bigint,
                      yr_2011 bigint,
                      yr_2012 bigint,
                      yr_2013 bigint,
                      yr_2014 bigint,
                      yr_2015 bigint,
                      yr_2016 bigint,
                      PRIMARY KEY (company_id, indicator_id)
                   )
                """)

In [ ]:
import json

In [ ]:
with open('companies.json') as f_in:
    companies = json.load(f_in)

In [ ]:
type(companies)

In [ ]:
companies[0]

In [ ]:
type(companies[0])

In [ ]:
session.execute(
   """
   INSERT INTO company (company_id, name_latest, name_previous)
   VALUES (%s, %s, %s)
   """,
   ("1000045", "Nicholas Financial Inc", ""))

In [ ]:
result_set = session.execute("SELECT * FROM company")

In [ ]:
result_set.current_rows

In [ ]:
for company in companies:
    try:
        session.execute(
                           """
                           INSERT INTO company (company_id, name_latest, name_previous)
                           VALUES (%s, %s, %s)
                           """,
                           (company['company_id'],company['name_latest'], company['names_previous']))
    except:
        pass

In [ ]:
result_set = session.execute("SELECT * FROM company")

In [ ]:
result_set.current_rows

In [ ]:
def safe_int(str_val):
    try:
        if str_val == '':
            return(0)
        else:
            return(int(str_val))
    except:
        return(0)

In [ ]:
with open('indicators_by_company.csv') as f:
    # the first line is a list of column names and can be ignored.
    _ = f.readline()
    for indicator in f:
        ind_list = indicator.strip().split(",")
        session.execute(
         """
        INSERT INTO indicator_by_company (company_id, indicator_id, yr_2010, yr_2011, yr_2012, yr_2013, yr_2014, yr_2015, yr_2016)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (ind_list[0],ind_list[1],safe_int(ind_list[2]),safe_int(ind_list[3]),safe_int(ind_list[4]),safe_int(ind_list[5]),safe_int(ind_list[6]),safe_int(ind_list[7]),safe_int(ind_list[8]))
        )